In [10]:
import cv2
import numpy as np
import pickle
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

class DummyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(250 * 250 * 3, 784)
        self.fc2 = nn.Linear(784, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 6)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = DummyModel()

model.load_state_dict(torch.load('drive/MyDrive/new_model_weights.pth', weights_only=True, map_location=torch.device(device)))
LABELS = ['green', 'blue', 'orange', 'red', 'yellow', 'white']

def pred_from_contours(img: np.ndarray, model: nn.Module) -> list:
    # get bounding boxes of contours
    detected = img.copy()

    contours = get_contours(detected, return_all=False)

    yolo_labels = []
    if len(contours) == 0:
        return detected

    for i in range(len(contours)):
        r = cv2.boundingRect(contours[i])
        x, y, w, h = r
        cropped_im = detected[y: y+h, x: x+w]
        resized = cv2.resize(cropped_im, (250, 250))
        resized = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)
        resized = torch.from_numpy(resized).float()
        resized = resized.unsqueeze(0)
        pred = model(resized)
        pred = F.softmax(pred, dim=1)
        pred = torch.argmax(pred, dim=1)
        pred = pred.item()
        center_x = x + (w // 2)
        center_y = y + (h // 2)
        norm_x = center_x / 500
        norm_y = center_y / 500
        norm_w = w / 500
        norm_h = h / 500
        yolo_labels.append((pred, norm_x, norm_y, norm_w, norm_h))

    return yolo_labels
# data dir
data_dir = Path('data')

# COLOR RANGES

# HSV
GREEN = [[40, 50, 50], [70, 255, 255]]

# HSV
BLUE = [[90, 100, 100], [140, 255, 255]]

# HSV
ORANGE = [[5, 100, 100], [25, 255, 255]]

# HSV BUT USE RGB
RED = [[115, 100, 100], [140, 255, 255]]

# HSV
YELLOW = [[25, 110, 110], [50, 255, 255]]

# HLS
WHITE = [[0, 160, 0], [179, 255, 255]]

RANGES = [GREEN, BLUE, ORANGE, RED, YELLOW, WHITE]


def detect_color(img: np.ndarray, color_range: list, color_space=cv2.COLOR_BGR2HSV) -> np.ndarray:
    image = img.copy()
    original = image.copy()

    image = cv2.cvtColor(image, color_space)

    lower = np.array(color_range[0], dtype="uint8")
    upper = np.array(color_range[1], dtype="uint8")

    # Create a mask, erode and dilate to remove noise
    mask = cv2.inRange(image, lower, upper)
    mask = cv2.erode(mask, None, iterations=4)
    mask = cv2.dilate(mask, None, iterations=4)

    detected = cv2.bitwise_and(original, original, mask=mask)

    return detected

def filter_color(img: np.ndarray, color_range: list, color_space=cv2.COLOR_BGR2HSV) -> np.ndarray:
    detected = detect_color(img, color_range, color_space)

    contours, heirarchy = get_contours(detected)

    if len(contours) == 0:
        return detected

    vis = np.zeros((detected.shape[0], detected.shape[1]), dtype=np.uint8)

    for i in range(len(heirarchy[0])):
        r = cv2.boundingRect(contours[i])
        if heirarchy[0][i][2] > -1:
            epsilon = 0.1 * cv2.arcLength(contours[i], True)
            approx = cv2.approxPolyDP(contours[i], epsilon, True)
            if len(approx) == 4:
                cv2.drawContours(vis, contours, i, (255, 255, 255), -1)

    new_vis = cv2.bitwise_and(detected, detected, mask=vis)

    return new_vis

def get_contours(img: np.ndarray, thresholds=[50,200], return_all=True, approx_poly=False) -> np.ndarray:
    blurred = cv2.GaussianBlur(img, (5, 5), 0)

    thresholded = cv2.threshold(cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY), 25, 255, cv2.THRESH_BINARY)[1]

    canny = cv2.Canny(thresholded, thresholds[0], thresholds[1])

    contours, heirarchy = cv2.findContours(canny, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)

    if return_all:
        return (contours, heirarchy)
    else:
        kept_contours = []
        for i in range(len(heirarchy[0])):
            if heirarchy[0][i][2] > -1:
                kept_contours.append(contours[i])
        kept_contours = sorted(kept_contours, key=cv2.contourArea, reverse=True)
        if len(kept_contours) > 9:
            kept_contours = kept_contours[:9]

        return kept_contours

def get_labels(fp):
  cropped = cv2.imread(fp)
  cropped = cv2.resize(cropped, (500, 500))

  final_image = np.zeros_like(cropped)

  for i, color in enumerate(RANGES):
      if i == 3:
          # RED
          detected = filter_color(cropped, color, cv2.COLOR_RGB2HSV)
      elif i == 5:
          # WHITE
          detected = filter_color(cropped, color, cv2.COLOR_BGR2HLS)
      else:
          detected = filter_color(cropped, color)
      final_image = cv2.bitwise_or(final_image, detected)


  pred = pred_from_contours(final_image, model)
  return pred

def main():


    # cropped = cv2.imread('simulated_cube_dataset/images/train/0313.jpg')
    # x1, y1, x2, y2 = [350, 276, 594, 491]
    # cropped = cropped[y1:y2, x1:x2]
    # cropped = cv2.imread('data/sim2.png')

    # cropped = cv2.imread()
    # cropped = cv2.resize(cropped, (500, 500))

    # final_image = np.zeros_like(cropped)

    # for i, color in enumerate(RANGES):
    #     if i == 3:
    #         # RED
    #         detected = filter_color(cropped, color, cv2.COLOR_RGB2HSV)
    #     elif i == 5:
    #         # WHITE
    #         detected = filter_color(cropped, color, cv2.COLOR_BGR2HLS)
    #     else:
    #         detected = filter_color(cropped, color)
    #     final_image = cv2.bitwise_or(final_image, detected)


    # pred = pred_from_contours(final_image, model)

    pred = get_labels('drive/MyDrive/isolated_cube_dataset/train/images/0015.jpg')

    print(pred)
    print(len(pred))
    for label in pred:
      print(" ".join(map(str, label)))

main()


[(0, 0.76, 0.262, 0.282, 0.274), (0, 0.174, 0.356, 0.282, 0.26), (2, 0.466, 0.312, 0.274, 0.254), (2, 0.802, 0.538, 0.268, 0.246), (2, 0.514, 0.58, 0.268, 0.248), (0, 0.56, 0.832, 0.268, 0.242), (5, 0.224, 0.62, 0.262, 0.234), (4, 0.278, 0.876, 0.258, 0.228), (5, 0.842, 0.794, 0.252, 0.23)]
9
0 0.76 0.262 0.282 0.274
0 0.174 0.356 0.282 0.26
2 0.466 0.312 0.274 0.254
2 0.802 0.538 0.268 0.246
2 0.514 0.58 0.268 0.248
0 0.56 0.832 0.268 0.242
5 0.224 0.62 0.262 0.234
4 0.278 0.876 0.258 0.228
5 0.842 0.794 0.252 0.23


In [15]:
image_fp = 'drive/MyDrive/isolated_cube_dataset/{}/images/'
label_fp = 'drive/MyDrive/isolated_cube_dataset/{}/labels/'

def determine_split(im_processed: int):
  '''
  takes in integer value, returns split and upper bound
  '''
  if im_processed < 800:
    return 'train', 799
  elif 800 <= im_processed < 900:
    return 'val', 899
  else:
    return 'test', 1000

def determine_batch(curr, upper_bound):
  """
  checks if 20 will overflow current split, returns proper value
  """
  next_batch = curr + 50
  res = upper_bound - next_batch
  if res < 0:
    return 50 + res
  else:
    return 50


def get_fns(cur, batch):
  return ["{:04d}".format(i) for i in range(cur, cur + batch + 1)]

def write_labels(split, fn, labels):
  pt = Path(label_fp.format(split))
  path = pt / (fn + '.txt')
  with open(path, 'w') as fp:
    for label in labels:
      res = " ".join(map(str, label)) + '\n'
      fp.write(res)


In [21]:
import os
cur = 800

while cur <= 1000:
  split, upper_bound = determine_split(cur)
  batch = determine_batch(cur, upper_bound)
  fns = get_fns(cur, batch)
  print(f'labeling {cur}-{cur+batch} images in {split} split')
  for fn in fns:
    fp = image_fp.format(split) + fn +'.jpg'
    isExist = os.path.exists(fp)
    if not isExist:
      continue
    labels = get_labels(fp)
    write_labels(split, fn, labels)
  print(f'labeled')
  if not batch:
    cur += 1
  cur += batch



labeling 800-850 images in val split
labeled
labeling 850-899 images in val split
labeled
labeling 899-899 images in val split
labeled
labeling 900-950 images in test split
labeled
labeling 950-1000 images in test split
labeled
labeling 1000-1000 images in test split
labeled
